# Deliverable 3: Optimizing the Outcomes

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from pathlib import Path
import kerastuner as kt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

c:\Users\justi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  if __name__ == '__main__':


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# What percentage of donations are acutally used for something? 
successes = len(application_df[application_df.IS_SUCCESSFUL == 1])
total = len(application_df)
successes/total

0.5324061926003674

## Preprocessing

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df.APPLICATION_TYPE.value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type_counts[app_type_counts < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# Determine which values to replace if counts are under ...  (Initial analysis was under 200)
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Adjust Income Amounts
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [10]:
# Create Income Amount Bins
def income_bin(row):
    income = row['INCOME_AMT']
    new_income = 0
    if income == '0':
        new_income = 0
    elif income == '1-9999':
        new_income = 10000
    elif income == '10000-24999':
        new_income = 25000
    elif income == '25000-99999':
        new_income = 100000
    elif income == '100000-499999':
        new_income = 500000
    elif income == '1M-5M':
        new_income = 1000000
    elif income == '5M-10M':
        new_income = 5000000
    elif income == '10M-50M':
        new_income = 10000000
    elif income == '50M+':
        new_income = 50000000
    return new_income

application_df.INCOME_AMT = application_df.apply(income_bin, axis=1)

In [11]:
def considerations_encode(row):
    consideration = row['SPECIAL_CONSIDERATIONS']
    new_consideration = 0
    if consideration == 'N':
        new_consideration = 0
    elif consideration == 'Y':
        new_consideration = 1
    return new_consideration


application_df.SPECIAL_CONSIDERATIONS = application_df.apply(considerations_encode, axis=1)


In [12]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,10000,0,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,25000,0,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,500000,0,142590,1


In [13]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                 int64
SPECIAL_CONSIDERATIONS     int64
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [14]:
# Evaluate and Change Ask Types
min_ask = application_df.ASK_AMT.min()
max_ask = application_df.ASK_AMT.max()
median_ask = application_df.ASK_AMT.median()

print(f'Min: {min_ask}, Median: {median_ask}, Max: {max_ask}')


Min: 5000, Median: 5000.0, Max: 8597806340


In [15]:
# Bin ther ask amounts 
# Apply the bins, labeling with the arbitrary 1-5 categories for the network learning
bins = [0, 5000, 10000, 50000, 100000, 500000, 1000000, max_ask]
labels = [0, 1, 2, 3, 4, 5, 6]

application_df["ASK_BIN"] = pd.cut(
    application_df["ASK_AMT"], bins=bins, labels=labels)

application_df[["ASK_AMT","ASK_BIN"]].head(10)

,ASK_AMT,ASK_BIN
0,5000,0
1,108590,4
2,5000,0
3,6692,1
4,142590,4
5,5000,0
6,31452,2
7,7508025,6
8,94389,3
9,5000,0


In [16]:
median = application_df.groupby(['ASK_BIN'])['ASK_AMT'].median()
median

ASK_BIN
0       5000.0
1       7172.0
2      25948.5
3      71017.0
4     202566.0
5     697265.0
6    3527276.0
Name: ASK_AMT, dtype: float64

In [17]:
def asking_bin(row):
    bin = row['ASK_BIN']
    bin_mean = 0
    if bin == 0:
        bin_mean = int(median[0])
    elif bin == 1:
        bin_mean = int(median[1])
    elif bin == 2:
        bin_mean = int(median[2])
    elif bin == 3:
        bin_mean = int(median[3])
    elif bin == 4:
        bin_mean = int(median[4])
    elif bin == 5:
        bin_mean = int(median[5])
    elif bin == 6:
        bin_mean = int(median[6])
    return bin_mean


application_df.ASK_BIN = application_df.apply(asking_bin, axis=1)
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_BIN
0,T10,Independent,C1000,ProductDev,Association,1,0,0,5000,1,5000
1,T3,Independent,C2000,Preservation,Co-operative,1,10000,0,108590,1,202566
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,5000,0,5000
3,T3,CompanySponsored,C2000,Preservation,Trust,1,25000,0,6692,1,7172
4,T3,Independent,C1000,Heathcare,Trust,1,500000,0,142590,1,202566


In [18]:
application_df.drop(columns='ASK_AMT', inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_BIN
0,T10,Independent,C1000,ProductDev,Association,1,0,0,1,5000
1,T3,Independent,C2000,Preservation,Co-operative,1,10000,0,1,202566
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,0,0,5000
3,T3,CompanySponsored,C2000,Preservation,Trust,1,25000,0,1,7172
4,T3,Independent,C1000,Heathcare,Trust,1,500000,0,1,202566


In [19]:
# Encode the categorical variables
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()

In [20]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
# Merge one-hot encoded features and drop the originals
application_enc = application_df.merge(encode_df, left_index=True, right_index=True)
application_enc.drop(columns=application_cat, inplace=True)

In [22]:
application_enc.head()

,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_BIN,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1,0,0,1,5000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1,10000,0,1,202566,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1,0,0,0,5000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,1,25000,0,1,7172,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1,500000,0,1,202566,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
def df_trainer(df):
    # Remove the outcome and set up the training model
    y = df.IS_SUCCESSFUL
    X = df.drop(columns="IS_SUCCESSFUL")

    # Split the preprocessed data into a training and testing dataset, values are not balanced - apply stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=512, stratify=y)

    # Scale the data
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test


In [24]:
X_train_scaled, X_test_scaled, y_train, y_test = df_trainer(application_enc)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_layer1 = 80
hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# Layers
nn.add(tf.keras.layers.Dense(units=hidden_layer1,
       input_dim=number_input_features, activation='relu'))
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Define Checkpoint Path and filenames

os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2800      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,261
Trainable params: 5,261
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5
)


In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10, callbacks=[cp_callback])


Epoch 1/10
  1/804 [..............................] - ETA: 7:26 - loss: 0.7124 - accuracy: 0.5625
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 15/804 [..............................] - ETA: 3s - loss: 0.6658 - accuracy: 0.6271  
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 30/804 [>.............................] - ETA: 2s - loss: 0.6587 - accuracy: 0.6375
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 49/804 [>.............................] - ETA: 2s - loss: 0.6416 - accuracy: 0.6633
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5



In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5619 - accuracy: 0.7268 - 268ms/epoch - 999us/step
Loss: 0.5618887543678284, Accuracy: 0.7267638444900513


# Adjusting the Model

In [28]:
from re import X


tests = pd.DataFrame(columns=["Hidden_Layers", "Nodes", "Activation", "Accuracy"])
hidden_layers = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5]
nodes = ['150',
         '150', 
         '150',
         '56/76',
         '56/76',
         '56/76',
         '21/66/91',
         '21/66/91',
         '21/66/91',
         '80/30/80',
         '56/76/36/21',
         '80/60/40/20',
         '56/76/36/21/36',
         '56/76/36/21/36',
         '56/76/36/21/11',
         '56/76/36/21/11',
         '21/66/91/20/30']
activation = ['relu',
            'tanh',
            'softmax',
            'relu',
            'tanh',
            'softmax/relu',
            'tanh',
            'relu',
            'relu',
            'relu', 
            'softmax', 
            'relu',
            'relu',
            'softmax/relu/relu/relu/relu',
            'softmax/relu/relu/sigmoid/sigmoid',
            'softmax',
            'relu'
            ]
accuracy = [72.4,
            72.4,
            72.6,
            72.7,
            72.7,
            72.5,
            72.8, 
            72.7,
            72.7,
            72.4, 
            72.6,
            72.5,
            72.3,
            72.7,
            72.7,
            72.4,
            72.6
            ]

In [29]:
tests.Hidden_Layers = hidden_layers
tests.Nodes = nodes
tests.Activation = activation
tests.Accuracy = accuracy
tests

,Hidden_Layers,Nodes,Activation,Accuracy
0,1,150,relu,72.4
1,1,150,tanh,72.4
2,1,150,softmax,72.6
3,2,56/76,relu,72.7
4,2,56/76,tanh,72.7
5,2,56/76,softmax/relu,72.5
6,3,21/66/91,tanh,72.8
7,3,21/66/91,relu,72.7
8,3,21/66/91,relu,72.7
9,3,80/30/80,relu,72.4


In [30]:
# Adding a third hidden layer, layers are 80/40/20, accuracy is .7225
# Trying Reverse pyramid relu, layers are 20/40/80, accuracy is .7251
# Trying from Tuner, 21/66/91 relu, accuracy is .7265
# Trying 22/66/91 tanh with 10 epochs, accuracy is .7264
# Trying the 22/66/91 tanh with 30 epochs, accuracy was .7258
# Trying 5 layers, based on tuner 56/76/36/21/36 with 10 epochs all relu, accuracy was .7230
# Trying 5 layers, based on 10/10/10/10/10 with 10 epochs all relu, accuracy was .7234
# Trying 5 layers, based on tuner 21/66/91/20/30 with 10 epochs all relu, accuracy was .7255
# Trying 5 layers, based on tuner 21/66/91/20/30 with 10 epochs all relu, accuracy was .7245
# Trying 3 layers, 21/66/91 relu/tanh/tanh, accuracy is .7222
# Trying 3 layers, 21/66/91 tanh/relu/relu, accuracy is .7254
# Trying 3 layers, 21/66/91 all exponential, acc is .4676
# Trying 3 layers, 21/66/91 all selu, acc is .7219
# Trying 3 layers, 88/88/88 all relu, acc is .7267
# Trying 3 layers, 80/30/4 all relu, acc is .7256
# Trying 3 layers, 8/3/4 all relu, acc is .7243
# Trying 3 layers, 80/30/80 all relu, acc is .7238
# Trying 4 layers, 80/60/40/20 all relu, acc is .7248
# Trying 5 layers, 80/60/40/20/10 all relu, acc is .7262
# Trying 5 layers, 80/60/40/20/10 all tanh, acc is .7249
# Trying 5 layers, 80/60/40/20/10 all softmax, acc is .7249
# Trying 4 layers, based on tuner 56/76/36/21 with 10 epochs all softmax, accuracy was .7258
# Trying 5 layers, based on tuner 56/76/36/21/36 with 10 epochs all softmax, accuracy was .5234
# Trying 5 layers, based on tuner 56/76/36/21/11 with 10 epochs all softmax, accuracy was .7243
# Trying 5 layers, based on tuner 56/76/36/21/11 with 10 epochs softmax/relu... , accuracy was .7270
# Trying 5 layers, based on tuner 56/76/36/21/11 with 10 epochs softmax/tanh... , accuracy was .7263
# Trying 3 layers, based on tuner 56/76/36 with 10 epochs softmax/tanh... , accuracy was .7265
# Trying 3 layers, based on tuner 56/76/36 with 10 epochs softmax/relu... , accuracy was .7264
# Trying 5 layers, based on tuner 56/76/36/21/11 with 10 epochs softmax/relu/relu/sigmoid/sigmoid , accuracy was .7272  ** Higest
# Trying 5 layers, based on tuner 56/76/36/21/11 with 10 epochs softmax/relu/relu/sigmoid/tanh , accuracy was .7244
# Trained 2 layers 56/76 softmax/relu, accuracy .7249
# Trained 2 layers 56/76 relu, accuracy .7270
# Trained 2 layers 56/76 tanh, accuracy .7271 * high and simple
# Trained 1 layer 150 tanh, acc .7243
# Trained 1 layer 150 relu, acc .7241
# Trained 1 layer 150 softmax, acc .7263
# Trying 5 layers, based on tuner 56/76/36/21/36 with 10 epochs softmax/relu..., accuracy was .7270









number_input_features = len(X_train_scaled[0])
hidden_layer1 = 56
hidden_layer2 = 76
hidden_layer3 = 36
hidden_layer4 = 21
hidden_layer5 = 11


nn = tf.keras.models.Sequential()

# Layers
nn.add(tf.keras.layers.Dense(units=hidden_layer1,
       input_dim=number_input_features, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='tanh'))
# nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='relu'))
# nn.add(tf.keras.layers.Dense(units=hidden_layer4, activation='relu'))
# nn.add(tf.keras.layers.Dense(units=hidden_layer5, activation='relu'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Define Checkpoint Path and filenames

os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 56)                1960      
                                                                 
 dense_4 (Dense)             (None, 76)                4332      
                                                                 
 dense_5 (Dense)             (None, 1)                 77        
                                                                 
Total params: 6,369
Trainable params: 6,369
Non-trainable params: 0
_________________________________________________________________


In [31]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5
)

fit_model = nn.fit(X_train_scaled, y_train, epochs=10, callbacks=[cp_callback])


Epoch 1/10
  1/804 [..............................] - ETA: 4:53 - loss: 0.7836 - accuracy: 0.5312
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 15/804 [..............................] - ETA: 2s - loss: 0.6898 - accuracy: 0.5729  
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 35/804 [>.............................] - ETA: 2s - loss: 0.6461 - accuracy: 0.6402
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 50/804 [>.............................] - ETA: 2s - loss: 0.6242 - accuracy: 0.6656
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5



In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5593 - accuracy: 0.7233 - 249ms/epoch - 928us/step
Loss: 0.5592706203460693, Accuracy: 0.7232652902603149


# Random Forests

In [33]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
barf = BalancedRandomForestClassifier(n_estimators=100, random_state=512)

y = application_enc.IS_SUCCESSFUL
X = application_enc.drop(columns='IS_SUCCESSFUL')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=512, stratify=y)


# Fit the model
barf.fit(X_train, y_train)
# calculate accuracy
y_pred = barf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

# Random Forests outperforms the NN in a fraction of the time

0.7164947707401813

In [34]:
# In a previous analysis, removing the Use Cases and ask amounts yielded higher accuracy

# Read new
application_df = pd.read_csv("Resources/charity_data.csv")

# Drop names
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

# Bin the types
# Highest accuracies were either with 50 or 200 as the cutoff

replace_application = list(app_type_counts[app_type_counts < 50].index)
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(
        app, "Other")

# Bin the Classification
# Highest accuracy was with 200 as the cutoff

replace_class = list(classification_counts[classification_counts < 200].index)
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(
        cls, "Other")


# Remove the Ask Amount - this seems to be a bad predictor, even when binned
application_df = application_df.drop(columns="ASK_AMT")

# Encode the categorical variables
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist(
)

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)


# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(
    encode_df, left_index=True, right_index=True)
application_df.drop(columns=application_cat, inplace=True)



# Scale and Train the Data for Random Forests
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns='IS_SUCCESSFUL')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=512)


# Fit the model after removing ASK_AMT
barf.fit(X_train, y_train)
# calculate accuracy
y_pred = barf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.7328118077574778

In [35]:
# Removing the USE_CASE and ASK_AMT, acc = .7342
# Removing the USE_CASE using ASK_BIN, acc = .7322
# Removing USE_CASE, ASK_AMT, STATUS = .7341 * Consistently the highest
# Removing USE_CASE, ASK_AMT, STATUS, AFFILIATION .6523
# Removing USE_CASE, ASK_AMT, STATUS, ORGANIZATION .7310
# Removing USE_CASE, ASK_AMT, STATUS, CLASSIFICATION .7330
# Removing USE_CASE, ASK_AMT, STATUS, APPLICATION_TYPE  .7012
# Removing USE_CASE, ASK_AMT, STATUS, INCOME_AMT  .7286
# Removing USE_CASE, ASK_AMT, STATUS, APPLICATION_TYPE  .7012

# Read new
application_df = pd.read_csv("Resources/charity_data.csv")

# Drop names
application_df.drop(columns=['EIN', 'NAME', 'ASK_AMT', 'STATUS'], inplace=True)

# Bin the income amounts
application_df.INCOME_AMT = application_df.apply(income_bin, axis=1)

# Bin the asking amounts 
# application_df.ASK_BIN = application_df.apply(asking_bin, axis=1)

# Change Special considerations to numeric
application_df.SPECIAL_CONSIDERATIONS = application_df.apply(considerations_encode, axis=1)

# Bin the types
# Highest accuracies were either with 50 or 200 as the cutoff

replace_application = list(app_type_counts[app_type_counts < 200].index)
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(
        app, "Other")

# Bin the Classification
# Highest accuracy was with 200 as the cutoff

replace_class = list(classification_counts[classification_counts < 200].index)
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(
        cls, "Other")




# Encode the categorical variables
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist(
)

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)



# Merge one-hot encoded features and drop the originals
application_enc = application_df.merge(
    encode_df, left_index=True, right_index=True)
application_enc.drop(columns=application_cat, inplace=True)


# Scale and Train the Data for Random Forests
y = application_enc.IS_SUCCESSFUL
X = application_enc.drop(columns='IS_SUCCESSFUL')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=512)


# Fit the model after removing ASK_AMT
barf.fit(X_train, y_train)
# calculate accuracy
y_pred = barf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.731219591885099

### Try the NN model with these parameters

In [36]:
X_train_scaled, X_test_scaled, y_train, y_test = df_trainer(application_enc)

number_input_features = len(X_train_scaled[0])
hidden_layer1 = 21
hidden_layer2 = 66
hidden_layer3 = 91
# hidden_layer4 = 21
# hidden_layer5 = 11


nn = tf.keras.models.Sequential()

# Layers
nn.add(tf.keras.layers.Dense(units=hidden_layer1,
       input_dim=number_input_features, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='tanh'))
# nn.add(tf.keras.layers.Dense(units=hidden_layer4, activation='sigmoid'))
# nn.add(tf.keras.layers.Dense(units=hidden_layer5, activation='sigmoid'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Define Checkpoint Path and filenames

os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 21)                735       
                                                                 
 dense_7 (Dense)             (None, 66)                1452      
                                                                 
 dense_8 (Dense)             (None, 91)                6097      
                                                                 
 dense_9 (Dense)             (None, 1)                 92        
                                                                 
Total params: 8,376
Trainable params: 8,376
Non-trainable params: 0
_________________________________________________________________


In [37]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5
)

fit_model = nn.fit(X_train_scaled, y_train, epochs=20, callbacks=[cp_callback])


Epoch 1/20
  1/804 [..............................] - ETA: 5:41 - loss: 0.6778 - accuracy: 0.5625
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 15/804 [..............................] - ETA: 3s - loss: 0.6402 - accuracy: 0.6542  
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 30/804 [>.............................] - ETA: 3s - loss: 0.6107 - accuracy: 0.6948
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 45/804 [>.............................] - ETA: 3s - loss: 0.6006 - accuracy: 0.7035
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5571 - accuracy: 0.7268 - 255ms/epoch - 953us/step
Loss: 0.5571009516716003, Accuracy: 0.7267638444900513


# Comparing the OneHotEncoded Ask Bins vs. the Numerical Ask Bins

The above accuracy is for the numerical ask bins, read from a single column, though these numbers are just categories. Below, each category is treated categorically and encoded. 

In [39]:

# Read new
application_df = pd.read_csv("Resources/charity_data.csv")

# Drop names
application_df.drop(columns=['EIN', 'NAME', 'STATUS'], inplace=True)

# With the ASK_BINS containing 7 bins, the accuracy was .7194
bins = [0, 5000, 10000, 50000, 100000, 500000, 1000000, max_ask]
labels = ['ASK_0', 'ASK_1', 'ASK_2', 'ASK_3', 'ASK_4', 'ASK_5', 'ASK_6']


# Bin the income amounts
application_df.INCOME_AMT = application_df.apply(income_bin, axis=1)

# Bin the asking amounts
# application_df.ASK_BIN = application_df.apply(asking_bin, axis=1)

# Change Special considerations to numeric
application_df.SPECIAL_CONSIDERATIONS = application_df.apply(
    considerations_encode, axis=1)

# Bin the types
# Highest accuracies were either with 50 or 200 as the cutoff

replace_application = list(app_type_counts[app_type_counts < 200].index)
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(
        app, "Other")

# Bin the Classification
# Highest accuracy was with 200 as the cutoff

replace_class = list(classification_counts[classification_counts < 200].index)
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(
        cls, "Other")


# Apply the bins, labeling with the arbitrary 1-5 categories for the network learning
application_df["ASK_BIN"] = pd.cut(
    application_df["ASK_AMT"], bins=bins, labels=labels)

# Remove the Ask Amount - this seems to be a bad predictor, even when binned
application_df = application_df.drop(columns="ASK_AMT")

# Encode the categorical variables
application_cat = application_df.dtypes[(application_df.dtypes == 'object') | (
    application_df.dtypes == 'category')].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)



# Merge one-hot encoded features and drop the originals
application_enc = application_df.merge(
    encode_df, left_index=True, right_index=True)
application_enc.drop(columns=application_cat, inplace=True)




In [40]:
application_enc.head()

,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,ASK_BIN_ASK_0,ASK_BIN_ASK_1,ASK_BIN_ASK_2,ASK_BIN_ASK_3,ASK_BIN_ASK_4,ASK_BIN_ASK_5,ASK_BIN_ASK_6
0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25000,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,500000,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Random Forests

In [41]:
# Scale and Train the Data for Random Forests
y = application_enc.IS_SUCCESSFUL
X = application_enc.drop(columns='IS_SUCCESSFUL')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=512, stratify=y)


# Fit the model after removing ASK_AMT
barf.fit(X_train, y_train)
# calculate accuracy
y_pred = barf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.7174164807040448

## Neural Network

In [42]:
X_train_scaled, X_test_scaled, y_train, y_test = df_trainer(application_enc)

number_input_features = len(X_train_scaled[0])
hidden_layer1 = 56
hidden_layer2 = 76
hidden_layer3 = 36
hidden_layer4 = 21
hidden_layer5 = 11


nn = tf.keras.models.Sequential()

# Layers
nn.add(tf.keras.layers.Dense(units=hidden_layer1,
       input_dim=number_input_features, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='relu'))
nn.add(tf.keras.layers.Dense(units=hidden_layer4, activation='relu'))
nn.add(tf.keras.layers.Dense(units=hidden_layer5, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Define Checkpoint Path and filenames

os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 56)                2352      
                                                                 
 dense_11 (Dense)            (None, 76)                4332      
                                                                 
 dense_12 (Dense)            (None, 36)                2772      
                                                                 
 dense_13 (Dense)            (None, 21)                777       
                                                                 
 dense_14 (Dense)            (None, 11)                242       
                                                                 
 dense_15 (Dense)            (None, 1)                 12        
                                                                 
Total params: 10,487
Trainable params: 10,487
Non-trai

In [43]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5
)

fit_model = nn.fit(X_train_scaled, y_train, epochs=20, callbacks=[cp_callback])


Epoch 1/20
  1/804 [..............................] - ETA: 9:22 - loss: 0.6492 - accuracy: 0.6562
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 15/804 [..............................] - ETA: 3s - loss: 0.6618 - accuracy: 0.5917  
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 30/804 [>.............................] - ETA: 3s - loss: 0.6459 - accuracy: 0.6250
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 45/804 [>.............................] - ETA: 2s - loss: 0.6326 - accuracy: 0.6535
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - loss: 0.5570 - accuracy: 0.7241 - 265ms/epoch - 989us/step
Loss: 0.5569790005683899, Accuracy: 0.7240816354751587


# Final Model

In [45]:
# Read
application_df = pd.read_csv("Resources/charity_data.csv")

# Drop the name and ID
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

# Replace Values
replace_application = list(app_type_counts[app_type_counts < 50].index)
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

# Replace Classifications
replace_class = list(classification_counts[classification_counts < 200].index)
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(
        cls, "Other")

# Bin the Incomes
application_df.INCOME_AMT = application_df.apply(income_bin, axis=1)

# Encode the Special Considerations
application_df.SPECIAL_CONSIDERATIONS = application_df.apply(
    considerations_encode, axis=1)

# Bin the Asking Amounts
bins = [0, 5000, 10000, 50000, 100000, 500000, 1000000, max_ask]
labels = [0, 1, 2, 3, 4, 5, 6]

application_df["ASK_BIN"] = pd.cut(
    application_df["ASK_AMT"], bins=bins, labels=labels)

median = application_df.groupby(['ASK_BIN'])['ASK_AMT'].median()

application_df.ASK_BIN = application_df.apply(asking_bin, axis=1)
application_df.drop(columns='ASK_AMT', inplace=True)

# Encode the object data
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)

# Merge one-hot encoded features and drop the originals
application_enc = application_df.merge(encode_df, left_index=True, right_index=True)
application_enc.drop(columns=application_cat, inplace=True)


# Split into Training and Testing Sets
X_train_scaled, X_test_scaled, y_train, y_test = df_trainer(application_enc)



In [46]:
# Neural Network Model
number_input_features = len(X_train_scaled[0])
hidden_layer1 = 100
hidden_layer2 = 50
# hidden_layer3 = 91
# hidden_layer4 = 21
# hidden_layer5 = 11


nn = tf.keras.models.Sequential()

# Layers
nn.add(tf.keras.layers.Dense(units=hidden_layer1,
       input_dim=number_input_features, activation='relu'))
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='relu'))
# nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='tanh'))
# nn.add(tf.keras.layers.Dense(units=hidden_layer4, activation='tanh'))
# nn.add(tf.keras.layers.Dense(units=hidden_layer5, activation='tanh'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Define Checkpoint Path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 100)               4500      
                                                                 
 dense_17 (Dense)            (None, 50)                5050      
                                                                 
 dense_18 (Dense)            (None, 1)                 51        
                                                                 
Total params: 9,601
Trainable params: 9,601
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Compile the Neural Network
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5
)

fit_model = nn.fit(X_train_scaled, y_train, epochs=10, callbacks=[cp_callback])


# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/10
  1/804 [..............................] - ETA: 5:07 - loss: 0.6286 - accuracy: 0.7812
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 20/804 [..............................] - ETA: 2s - loss: 0.6517 - accuracy: 0.6578  
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 35/804 [>.............................] - ETA: 2s - loss: 0.6341 - accuracy: 0.6732
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5
 50/804 [>.............................] - ETA: 2s - loss: 0.6227 - accuracy: 0.6800
Epoch 1: saving model to checkpoints\weights.01.hdf5

Epoch 1: saving model to checkpoints\weights.01.hdf5

